# Midterm <a class="tocSkip">
    
Its midterm season Wizards! The midterm is to make a bipedal humanoid robot walk in a simulation. You can use OpenAI Gym for the environment, this [link](https://github.com/search?q=bipedal+gym) shows some potential solutions that you can use to help you when you build your own. Submit your repository to schoolofaigrading@gmail.com . We will review your work and send back grades! We’re looking for good documentation, readable code, and bonus points for using reinforcement learning in a novel way for this challenge. Due date for all midterms is October 29, 2018 at 12 PM PST. Good luck!

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Exercise" data-toc-modified-id="Exercise-1">Exercise</a></span></li><li><span><a href="#Report" data-toc-modified-id="Report-2">Report</a></span></li></ul></div>

# Exercise

# Report